In [1]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz

import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml

In [2]:
df = pd.read_csv('Data.csv')

In [3]:
df.head()

,beer/ABV,beer/name,beer/style,review/overall,review/text,review/timeUnix,user/profileName,Binary,UserIndex,ItemIndex,review,conca_review,keyVector,keyphrases_indices_length
0,5.5,Caldera Pale Ale,American Pale Ale (APA),4.0,Caldera- Pale Ale\t\tPicked up at Grape & Gour...,1251327677,NJpadreFan,0,1767,659,"['caldera', 'pale', 'ale', 'picked', 'grape', ...",caldera pale ale picked grape gourmet vb ligh...,"[1, 7, 12, 14, 16, 29, 31, 34, 39, 40, 42, 45]",12
1,5.5,Caldera Pale Ale,American Pale Ale (APA),2.5,12 oz. can poured into a pint glass. I really ...,1250928902,vacax,0,6159,659,"['oz', 'poured', 'pint', 'glass', 'really', 'l...",oz poured pint glass really like beer like pa...,"[1, 16, 18, 20, 24, 39, 42, 43, 45, 55]",10
2,5.5,Caldera Pale Ale,American Pale Ale (APA),4.0,A: Pours a cloudy orange with a crisp white tw...,1249866208,d0ggnate,0,3516,659,"['pours', 'cloudy', 'orange', 'crisp', 'white'...",pours cloudy orange crisp white two finger he...,"[12, 36, 39, 42]",4
3,5.5,Caldera Pale Ale,American Pale Ale (APA),4.5,Received this can through Beer Powerball. I am...,1249847121,babyhobbes,1,2893,659,"['received', 'beer', 'powerball', 'really', 's...",received beer powerball really sorry nt know ...,"[12, 42]",2
4,5.5,Caldera Pale Ale,American Pale Ale (APA),4.5,355ml can bought from Beersofeurope.com...\t\t...,1249556277,mdagnew,1,4858,659,"['bought', 'beersofeurope', 'com', 'poured', '...",bought beersofeurope com poured golden yellow...,"[1, 5, 7, 12, 14, 16, 18, 19, 26, 31, 34, 40, ...",18


In [7]:
print len(df)
print len(df_train)

1002769
641773


In [4]:
df_train = pd.read_csv('Train.csv')

In [5]:
df_train.head()

,beer/ABV,beer/name,beer/style,review/overall,review/text,review/timeUnix,user/profileName,Binary,UserIndex,ItemIndex,review,conca_review,keyVector,keyphrases_indices_length
0,5.5,Caldera Pale Ale,American Pale Ale (APA),4.0,Caldera- Pale Ale\t\tPicked up at Grape & Gour...,1251327677,NJpadreFan,0,1767,659,"['caldera', 'pale', 'ale', 'picked', 'grape', ...",caldera pale ale picked grape gourmet vb ligh...,"[1, 7, 12, 14, 16, 29, 31, 34, 39, 40, 42, 45]",12
1,5.5,Caldera Pale Ale,American Pale Ale (APA),2.5,12 oz. can poured into a pint glass. I really ...,1250928902,vacax,0,6159,659,"['oz', 'poured', 'pint', 'glass', 'really', 'l...",oz poured pint glass really like beer like pa...,"[1, 16, 18, 20, 24, 39, 42, 43, 45, 55]",10
2,5.5,Caldera Pale Ale,American Pale Ale (APA),4.0,A: Pours a cloudy orange with a crisp white tw...,1249866208,d0ggnate,0,3516,659,"['pours', 'cloudy', 'orange', 'crisp', 'white'...",pours cloudy orange crisp white two finger he...,"[12, 36, 39, 42]",4
3,5.5,Caldera Pale Ale,American Pale Ale (APA),4.5,Received this can through Beer Powerball. I am...,1249847121,babyhobbes,1,2893,659,"['received', 'beer', 'powerball', 'really', 's...",received beer powerball really sorry nt know ...,"[12, 42]",2
4,5.5,Caldera Pale Ale,American Pale Ale (APA),4.5,355ml can bought from Beersofeurope.com...\t\t...,1249556277,mdagnew,1,4858,659,"['bought', 'beersofeurope', 'com', 'poured', '...",bought beersofeurope com poured golden yellow...,"[1, 5, 7, 12, 14, 16, 18, 19, 26, 31, 34, 40, ...",18


In [12]:
print df['review/text'][1]
print df['review'][1]

12 oz. can poured into a pint glass. I really like cans for beers like pale ales that are not meant for cellaring and am glad to have another one available in SoCal. Cool green tab but sadly no canning or best by date.		Pours a nice rich gold with a pure white head that grows excessively. Slow pour down a tilted glass and cannot even get the whole can into a pint glass. The retention is good, however, and lacing is great. Lots of bubbles cascading upwards from the bottom of the glass. The aroma is mild and consists of a bit of hoppy earthiness and some candied oranges.		The flavor starts with a mild hops bittering and quickly adds a touch of sweetness to balance it out. Earthy and slightly piney with bitter sappy sugar.. The bittering is not high, but there is little aroma hopping evident or much malt complexity either. This is a very light pale ale with little going on beyond some bittering hops, leaving a slightly floral but vegetal finish. An okay beer for a local brewpub, but not q